In [1]:
%load_ext autoreload
%load_ext autotime

time: 407 µs (started: 2023-03-12 13:14:57 -07:00)


In [2]:
import sys
sys.path.append("../")
import warnings
warnings.filterwarnings('ignore')
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

time: 1.41 s (started: 2023-03-12 13:14:58 -07:00)


In [3]:
import dask
import dask.array as da
import dask.bag as db
import dask.dataframe as dd
import json
import math
import numpy as np
import os
import pandas as pd
from dask import compute, delayed
from dask.dataframe import DataFrame
from dask.distributed import Client, LocalCluster, fire_and_forget, wait, worker_client
from wisio.utils.file_utils import ensure_dir
from wisio.utils.json_encoders import NpEncoder
from wisio.utils.logger import create_logger, format_log


time: 905 ms (started: 2023-03-12 13:14:59 -07:00)


In [4]:
client = Client(LocalCluster(n_workers=32, local_directory=f"/var/tmp/playground-quartz/local"))
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 32
Total threads: 64,Total memory: 125.50 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33404,Workers: 32
Dashboard: http://127.0.0.1:8787/status,Total threads: 64
Started: Just now,Total memory: 125.50 GiB
Comm: tcp://127.0.0.1:39850,Total threads: 2
Dashboard: http://127.0.0.1:33422/status,Memory: 3.92 GiB
Nanny: tcp://127.0.0.1:36929,


time: 4.72 s (started: 2023-03-12 13:15:01 -07:00)


In [5]:
import json

def load_global_min_max(log_dir: str):
    with open(f"{log_dir}/global.json") as file:
        global_min_max = json.load(file)
    return global_min_max

time: 7.34 ms (started: 2023-03-12 13:15:09 -07:00)


In [6]:
DELTA = 0.0001


time: 744 µs (started: 2023-03-12 13:15:10 -07:00)


In [96]:
# log_dir = "/p/vast1/iopp/recorder_app_logs/cm1/nodes-32/workflow-4/_parquet"
# log_dir = "/p/vast1/iopp/recorder_app_logs/hacc/nodes-32/workflow-0/_parquet"
log_dir = "/p/vast1/iopp/recorder_app_logs/montage/nodes-32/_parquet"
ddf = dd.read_parquet(f"{log_dir}/*.parquet")
# ddf = dd.read_parquet(f"{log_dir}/*.parquet", engine='pyarrow', filters=[[('file_id', '=', 4531734247156679037)], [('file_id', '=', 4531734247515089012)]])
ddf

,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,hostname,app,proc_name,file_name,size,acc_pat,bandwidth,duration,tmid,file_id,proc_id
npartitions=12,,,,,,,,,,,,,,,,,,,,,
,int64,int64,int32,int32,int32,int32,float32,float32,object,int32,object,object,object,object,int64,int32,float32,float32,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


time: 150 ms (started: 2023-03-12 13:36:22 -07:00)


In [97]:
ddf.head()

,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,...,app,proc_name,file_name,size,acc_pat,bandwidth,duration,tmid,file_id,proc_id
0,1,111084,0,314672,0,3,0.242405,0.242414,__xstat64,0,...,mImgtbl,mImgtbl#lassen247#111084#35184372403504,/p/gpfs1/iopp/temp/montage/2854964/2,0,0,0.0,0.000009,2424094,1571755611461267775,-4245390515581891276
1,2,111084,0,314672,0,3,0.242502,0.242732,fopen64,0,...,mImgtbl,mImgtbl#lassen247#111084#35184372403504,/p/gpfs1/iopp/temp/montage/2854964/2/run/Kimag...,0,0,0.0,0.000230,2426168,7638324191939094055,-4245390515581891276
2,3,111084,0,314672,0,3,0.242851,0.242868,opendir,0,...,mImgtbl,mImgtbl#lassen247#111084#35184372403504,/p/gpfs1/iopp/temp/montage/2854964/2,0,0,0.0,0.000017,2428593,1571755611461267775,-4245390515581891276
3,4,111084,0,314672,0,3,0.242946,0.242956,__xstat64,0,...,mImgtbl,mImgtbl#lassen247#111084#35184372403504,/p/gpfs1/iopp/temp/montage/2854964/2,0,0,0.0,0.000010,2429507,1571755611461267775,-4245390515581891276
4,5,111084,0,314672,0,3,0.243006,0.243016,__xstat64,0,...,mImgtbl,mImgtbl#lassen247#111084#35184372403504,/p/gpfs1/iopp/temp/montage/2854964,0,0,0.0,0.000010,2430107,5347462497940582382,-4245390515581891276


time: 2.6 s (started: 2023-03-12 13:36:23 -07:00)


In [98]:
global_min_max = load_global_min_max(log_dir=log_dir)
global_min_max

{'file_id': [113104469769545699, 8946970060350424228],
 'proc_id': [-7874823657267864124, 8238629349086124484],
 'tmid': [0, 1752590553]}

time: 5.55 ms (started: 2023-03-12 13:36:26 -07:00)


In [99]:
from dask.distributed import Future, get_client
from typing import Union

DELTA_BINS = [
    0,
    0.001,
    0.01,
    0.1,
    0.25,
    0.5,
    0.75,
    1
]
DELTA_BIN_LABELS = [
    'none',
    'trivial',
    'very low',
    'low',
    'medium',
    'high',
    'very high',
    'critical'
]
TIME_PRECISION = 1e7

def set_metric_deltas(df: pd.DataFrame, metric: str, max_io_time: float):
    metric_col, csp_col, delta_col = (
        f"{metric}_sum",
        f"{metric}_csp",
        f"{metric}_delta",
    )
    df[csp_col] = df[metric_col].cumsum() / max_io_time
    df[delta_col] = df[csp_col].diff().fillna(df[csp_col])
    return df


def set_metric_percentages(df: pd.DataFrame, metric: str, max_io_time: float):
    metric_col, pero_col, perr_col = (
        f"{metric}_sum",
        f"{metric}_pero",
        f"{metric}_perr"
    )
    df[pero_col] = df[metric_col] / max_io_time
    df[perr_col] = df[metric_col] / df[metric_col].sum()
    return df


def set_metric_scores(df: pd.DataFrame, metric: str, col: str):
    bin_col, score_col, threshold_col = (
        f"{metric}_bin",
        f"{metric}_score",
        f"{metric}_th",
    )
    df[bin_col] = np.digitize(df[col], bins=DELTA_BINS, right=True)
    df[score_col] = np.choose(df[bin_col] - 1, choices=DELTA_BIN_LABELS, mode='clip')
    df[threshold_col] = np.choose(df[bin_col] - 1, choices=DELTA_BINS, mode='clip')
    return df.drop(columns=[bin_col])

def compute_tranges(global_min_max: dict, precision=TIME_PRECISION):
    tmid_min, tmid_max = global_min_max['tmid']
    tranges = np.arange(tmid_min, tmid_max, precision)
    return get_client().scatter(tranges)


def set_tranges(df: pd.DataFrame, tranges: Union[Future, np.ndarray]):
    tranges = tranges.result() if isinstance(tranges, Future) else tranges
    return df.assign(trange=np.digitize(df['tmid'], bins=tranges, right=True))

time: 3.65 ms (started: 2023-03-12 13:36:27 -07:00)


In [100]:
tranges = compute_tranges(global_min_max=global_min_max)
tranges

<Future: finished, type: numpy.ndarray, key: ndarray-25d6b6e59e08cc3fd57ee56d864e2c1f>

time: 14.9 ms (started: 2023-03-12 13:36:28 -07:00)


In [101]:
VIEW_TYPES = ['trange', 'file_name', 'proc_name']

time: 493 µs (started: 2023-03-12 13:36:29 -07:00)


In [102]:
from typing import Iterable

def deepflatten(collection, ignore_types=(bytes, str)):
    for x in collection:
        if isinstance(x, Iterable) and not isinstance(x, ignore_types):
            yield from deepflatten(x)
        else:
            yield x

def unique_flatten():
    return dd.Aggregation(
        'unique_flatten',
        lambda s: s.apply(lambda x: np.unique(x).tolist()),
        lambda s0: s0.apply(lambda x: np.unique(x).tolist()),
        lambda s1: s1.apply(lambda x: np.unique(list(deepflatten(x))).tolist()),
    )

time: 1.23 ms (started: 2023-03-12 13:36:30 -07:00)


In [105]:
from copy import copy

view_types = copy(VIEW_TYPES)
view_types.append('io_cat')
view_types.append('acc_pat')

main_view = ddf[(ddf['cat'] == 0) & (ddf['io_cat'].isin([1,2,3]))]  \
    .map_partitions(set_tranges, tranges=tranges) \
    .groupby(view_types) \
    .agg({
        'duration': [sum],
        'index': ['count'],
        'size': [min, max, sum],
        'file_name': ['first'],
        'proc_name': ['first'],
        'func_id': [unique_flatten()],
    }) \
    .persist()

wait(main_view)


DoneAndNotDoneFutures(done={<Future: finished, type: pandas.core.frame.DataFrame, key: ('aggregate-agg-ff793b3b1ca5232218cf0e5d7f22c7d8', 0)>}, not_done=set())

time: 9.39 s (started: 2023-03-12 13:37:07 -07:00)


In [106]:
len(main_view)

12211

time: 45 ms (started: 2023-03-12 13:37:16 -07:00)


In [107]:
from copy import copy

view_types = copy(VIEW_TYPES)
view_types.append('io_cat')
view_types.append('acc_pat')
view_types.append('func_id')

main_view = ddf[(ddf['cat'] == 0) & (ddf['io_cat'].isin([1,2,3]))]  \
    .map_partitions(set_tranges, tranges=tranges) \
    .groupby(view_types) \
    .agg({
        'duration': [sum],
        'index': ['count'],
        'size': [min, max, sum],
        # 'func_id': [unique_flatten()],
    }) \
    .persist()

wait(main_view)


DoneAndNotDoneFutures(done={<Future: finished, type: pandas.core.frame.DataFrame, key: ('aggregate-agg-17e97797a229462129108d58fd198d8f', 0)>}, not_done=set())

time: 4.96 s (started: 2023-03-12 13:37:17 -07:00)


In [108]:
len(main_view)

20371

time: 59.4 ms (started: 2023-03-12 13:37:22 -07:00)


In [109]:
main_view.head()

duration  \
                                                                                                                                 sum   
trange file_name                                          proc_name                               io_cat acc_pat func_id               
1      /p/gpfs1/iopp/temp/montage/2854964                 mImgtbl#lassen246#41262#35184372403504  3      0       __xstat64  0.000010   
                                                          mImgtbl#lassen247#111084#35184372403504 3      0       __xstat64  0.000010   
                                                          mImgtbl#lassen378#29393#35184372403504  3      0       __xstat64  0.000009   
       /p/gpfs1/iopp/temp/montage/2854964/0               mProjExec#lassen70#61898#35184372402064 3      0       __xstat64  0.000009   
       /p/gpfs1/iopp/temp/montage/2854964/0/2mass-atla... mProjExec#lassen70#61898#35184372402064 1      0       fread      0.001227   

                                                                                                                           index  \
                                                                                                                           count   
trange file_name                                          proc_name                               io_cat acc_pat func_id           
1      /p/gpfs1/iopp/temp/montage/2854964                 mImgtbl#lassen246#41262#35184372403504  3      0       __xstat64     1   
                                                          mImgtbl#lassen247#111084#35184372403504 3      0       __xstat64     1   
                                                          mImgtbl#lassen378#29393#35184372403504  3      0       __xstat64     1   
       /p/gpfs1/iopp/temp/montage/2854964/0               mProjExec#lassen70#61898#35184372402064 3      0       __xstat64     1   
       /p/gpfs1/iopp/temp/montage/2854964/0/2mass-atla... mProjExec#lassen70#61898#35184372402064 1      0       fread       150   

                                                                                                                           size  \
                                                                                                                            min   
trange file_name                                          proc_name                               io_cat acc_pat func_id          
1      /p/gpfs1/iopp/temp/montage/2854964                 mImgtbl#lassen246#41262#35184372403504  3      0       __xstat64    0   
                                                          mImgtbl#lassen247#111084#35184372403504 3      0       __xstat64    0   
                                                          mImgtbl#lassen378#29393#35184372403504  3      0       __xstat64    0   
       /p/gpfs1/iopp/temp/montage/2854964/0               mProjExec#lassen70#61898#35184372402064 3      0       __xstat64    0   
       /p/gpfs1/iopp/temp/montage/2854964/0/2mass-atla... mProjExec#lassen70#61898#35184372402064 1      0       fread        2   

                                                                                                                                   \
                                                                                                                              max   
trange file_name                                          proc_name                               io_cat acc_pat func_id            
1      /p/gpfs1/iopp/temp/montage/2854964                 mImgtbl#lassen246#41262#35184372403504  3      0       __xstat64      0   
                                                          mImgtbl#lassen247#111084#35184372403504 3      0       __xstat64      0   
                                                          mImgtbl#lassen378#29393#35184372403504  3      0       __xstat64      0   
       /p/gpfs1/iopp/temp/montage/2854964/0               mProjExec#lassen70#61898#35184372402064 3      0       __xstat64      0  

time: 40.2 ms (started: 2023-03-12 13:37:25 -07:00)


In [110]:
main_view = main_view.reset_index()
main_view.columns = ['_'.join(tup).rstrip('_') for tup in main_view.columns.values]
main_view

,trange,file_name,proc_name,io_cat,acc_pat,func_id,duration_sum,index_count,size_min,size_max,size_sum
npartitions=1,,,,,,,,,,,
,int64,object,object,int64,int64,object,float32,int64,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...


time: 57.5 ms (started: 2023-03-12 13:37:27 -07:00)


In [111]:
wait(main_view)

DoneAndNotDoneFutures(done={<Future: finished, type: pandas.core.frame.DataFrame, key: ('aggregate-agg-17e97797a229462129108d58fd198d8f', 0)>}, not_done=set())

time: 11.4 ms (started: 2023-03-12 13:37:28 -07:00)


In [112]:
list(main_view[main_view['io_cat'] == 3]['func_id'].unique().compute())

['__xstat64',
 'fclose',
 'fopen64',
 'fseek',
 'opendir',
 'vfprintf',
 'unlink',
 'access',
 'close',
 'open',
 'remove']

time: 65.2 ms (started: 2023-03-12 13:37:29 -07:00)


In [126]:
main_view['read_count'] = 0
main_view['write_count'] = 0
main_view['metadata_count'] = 0
main_view['read_count'] = main_view['read_count'].mask(main_view['io_cat'] == 1, main_view['index_count'])
main_view['write_count'] = main_view['write_count'].mask(main_view['io_cat'] == 2,  main_view['index_count'])
main_view['metadata_count'] = main_view['metadata_count'].mask(main_view['io_cat'] == 3,  main_view['index_count'])

main_view['read_size'] = 0
main_view['write_size'] = 0
main_view['metadata_size'] = 0
main_view['read_size'] = main_view['read_size'].mask(main_view['io_cat'] == 1, main_view['size_sum'])
main_view['write_size'] = main_view['write_size'].mask(main_view['io_cat'] == 2,  main_view['size_sum'])
main_view['metadata_size'] = main_view['metadata_size'].mask(main_view['io_cat'] == 3,  main_view['size_sum'])

main_view['read_time'] = 0
main_view['write_time'] = 0
main_view['metadata_time'] = 0
main_view['read_time'] = main_view['read_time'].mask(main_view['io_cat'] == 1, main_view['duration_sum'])
main_view['write_time'] = main_view['write_time'].mask(main_view['io_cat'] == 2,  main_view['duration_sum'])
main_view['metadata_time'] = main_view['metadata_time'].mask(main_view['io_cat'] == 3,  main_view['duration_sum'])

main_view['sequential_count'] = 0
main_view['random_count'] = 0
main_view['sequential_count'] = main_view['sequential_count'].mask(main_view['acc_pat'] == 0, main_view['index_count'])
main_view['random_count'] = main_view['random_count'].mask(main_view['acc_pat'] == 1, main_view['index_count'])

main_view['sequential_size'] = 0
main_view['random_size'] = 0
main_view['sequential_size'] = main_view['sequential_size'].mask(main_view['acc_pat'] == 0, main_view['size_sum'])
main_view['random_size'] = main_view['random_size'].mask(main_view['acc_pat'] == 1, main_view['size_sum'])

main_view['sequential_time'] = 0
main_view['random_time'] = 0
main_view['sequential_time'] = main_view['sequential_time'].mask(main_view['acc_pat'] == 0, main_view['duration_sum'])
main_view['random_time'] = main_view['random_time'].mask(main_view['acc_pat'] == 1, main_view['duration_sum'])


main_view['data_count'] = main_view['write_count'] + main_view['read_count']
main_view['data_size'] = main_view['write_size'] + main_view['read_size']
main_view['data_time'] = main_view['write_time'] + main_view['read_time']

main_view['open_time'] = 0
main_view['close_time'] = 0
main_view['seek_time'] = 0
main_view['stat_time'] = 0
main_view['open_time'] = main_view['open_time'].mask(main_view['func_id'].str.contains('open') & ~main_view['func_id'].str.contains('dir'),  main_view['duration_sum'])
main_view['close_time'] = main_view['close_time'].mask(main_view['func_id'].str.contains('close') & ~main_view['func_id'].str.contains('dir'),  main_view['duration_sum'])
main_view['seek_time'] = main_view['seek_time'].mask(main_view['func_id'].str.contains('seek'),  main_view['duration_sum'])
main_view['stat_time'] = main_view['stat_time'].mask(main_view['func_id'].str.contains('stat'),  main_view['duration_sum'])

# main_view['workload_type'] = 0
# main_view['workload_type'] = main_view['workload_type'].mask(main_view['read_count'] == 0, 3)
# main_view['workload_type'] = main_view['workload_type'].mask(main_view['write_count'] == 0, 1)

# agg_dict = {col: sum for col in main_view.columns}
# agg_dict['file_name_first'] = 'first'
# agg_dict['proc_name_first'] = 'first'
# agg_dict['func_id_unique_flatten'] = unique_flatten()
# for view_type in VIEW_TYPES:
#     agg_dict.pop(view_type)

final_view = main_view \
    .groupby(VIEW_TYPES) \
    .sum() \
    .persist()

final_view


,io_cat,acc_pat,duration_sum,index_count,size_min,size_max,size_sum,read_count,write_count,metadata_count,read_size,write_size,metadata_size,read_time,write_time,metadata_time,sequential_count,random_count,sequential_size,random_size,sequential_time,random_time,data_count,data_size,data_time,open_time,close_time,seek_time,stat_time
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,int64,float32,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


time: 706 ms (started: 2023-03-12 13:40:42 -07:00)


In [127]:
wait(final_view)

DoneAndNotDoneFutures(done={<Future: finished, type: pandas.core.frame.DataFrame, key: ('dataframe-groupby-sum-agg-3f360e25091064369a751e366725aef6', 0)>}, not_done=set())

time: 641 ms (started: 2023-03-12 13:40:43 -07:00)


In [128]:
len(final_view.columns)

29

time: 3.44 ms (started: 2023-03-12 13:40:44 -07:00)


In [129]:
final_view.head()

io_cat  \
trange file_name                                          proc_name                                         
1      /p/gpfs1/iopp/temp/montage/2854964                 mImgtbl#lassen246#41262#35184372403504        3   
                                                          mImgtbl#lassen247#111084#35184372403504       3   
                                                          mImgtbl#lassen378#29393#35184372403504        3   
       /p/gpfs1/iopp/temp/montage/2854964/0               mProjExec#lassen70#61898#35184372402064       3   
       /p/gpfs1/iopp/temp/montage/2854964/0/2mass-atla... mProjExec#lassen70#61898#35184372402064      16   

                                                                                                   acc_pat  \
trange file_name                                          proc_name                                          
1      /p/gpfs1/iopp/temp/montage/2854964                 mImgtbl#lassen246#41262#35184372403504         0   
                                                          mImgtbl#lassen247#111084#35184372403504        0   
                                                          mImgtbl#lassen378#29393#35184372403504         0   
       /p/gpfs1/iopp/temp/montage/2854964/0               mProjExec#lassen70#61898#35184372402064        0   
       /p/gpfs1/iopp/temp/montage/2854964/0/2mass-atla... mProjExec#lassen70#61898#35184372402064        1   

                                                                                                   duration_sum  \
trange file_name                                          proc_name                                               
1      /p/gpfs1/iopp/temp/montage/2854964                 mImgtbl#lassen246#41262#35184372403504       0.000010   
                                                          mImgtbl#lassen247#111084#35184372403504      0.000010   
                                                          mImgtbl#lassen378#29393#35184372403504       0.000009   
       /p/gpfs1/iopp/temp/montage/2854964/0               mProjExec#lassen70#61898#35184372402064      0.000009   
       /p/gpfs1/iopp/temp/montage/2854964/0/2mass-atla... mProjExec#lassen70#61898#35184372402064      0.001487   

                                                                                                   index_count  \
trange file_name                                          proc_name                                              
1      /p/gpfs1/iopp/temp/montage/2854964                 mImgtbl#lassen246#41262#35184372403504             1   
                                                          mImgtbl#lassen247#111084#35184372403504            1   
                                                          mImgtbl#lassen378#29393#35184372403504             1   
       /p/gpfs1/iopp/temp/montage/2854964/0               mProjExec#lassen70#61898#35184372402064            1   
       /p/gpfs1/iopp/temp/montage/2854964/0/2mass-atla... mProjExec#lassen70#61898#35184372402064          172   

                                                                                                   size_min  \
trange file_name                                          proc_name                                           
1      /p/gpfs1/iopp/temp/montage/2854964                 mImgtbl#lassen246#41262#35184372403504          0   
                                                          mImgtbl#lassen247#111084#35184372403504         0   
                                                          mImgtbl#lassen378#29393#35184372403504          0   
       /p/gpfs1/iopp/temp/montage/2854964/0               mProjExec#lassen70#61898#35184372402064         0   
       /p/gpfs1/iopp/temp/montage/2854964/0/2mass-atla... mProjExec#lassen70#61898#35184372402064        -2   

                                                                                                   size_max  \
trange file_name                                          proc_name   

time: 50.7 ms (started: 2023-03-12 13:40:45 -07:00)


In [130]:
final_view['open_time'].sum().compute()

3.808836209675974

time: 169 ms (started: 2023-03-12 13:40:47 -07:00)


In [131]:
final_view['close_time'].sum().compute()

1.16335408908094

time: 39.3 ms (started: 2023-03-12 13:40:48 -07:00)


In [132]:
max_io_time = final_view.groupby(['proc_name']).agg({'duration_sum':sum})['duration_sum'].max()
max_io_time

dd.Scalar<series-..., dtype=float32>

time: 26.5 ms (started: 2023-03-12 13:40:52 -07:00)


In [133]:
DELTA = 0.0001
CUT = 0.5

time: 1.04 ms (started: 2023-03-12 13:40:53 -07:00)


In [134]:
import itertools as it

def _view_permutations(r: int):
    return it.permutations(VIEW_TYPES, r + 1)

metric = 'duration'
views = {}
for view_permutation in it.chain.from_iterable(map(_view_permutations, range(len(VIEW_TYPES)))):
    # Read types
    parent_type = view_permutation[:-1]
    view_type = view_permutation[-1]
    # Get parent view
    parent_view = views[parent_type] if parent_type in views else final_view
    # Create colum names
    metric_col, delta_col = f"{metric}_sum", f"{metric}_delta"
    # Check view type
    if view_type is not 'proc_name':
        # Compute `proc_id` view first
        group_view = parent_view \
            .groupby([view_type, 'proc_name']) \
            .agg({metric_col: sum}) \
            .groupby([view_type]) \
            .max()
    else:
        # Compute group view
        group_view = parent_view \
            .groupby([view_type]) \
            .agg({metric_col: sum})
    # Set metric scores
    group_view = group_view \
        .map_partitions(set_metric_deltas, metric=metric, max_io_time=max_io_time) \
        .query(f"{delta_col} > @delta", local_dict={'delta': DELTA})
    # Find filtered records
    view = parent_view.query(f"{view_type} in @indices", local_dict={'indices': group_view.index.unique()})
    # Return views
    views[view_permutation] = view

len(views)

15

time: 562 ms (started: 2023-03-12 13:40:54 -07:00)


In [135]:
for view_key, view in views.items():
    print(view_key, len(view))

('trange',) 5980
('file_name',) 5242
('proc_name',) 5936
('trange', 'file_name') 5242
('trange', 'proc_name') 5936
('file_name', 'trange') 5242
('file_name', 'proc_name') 5202
('proc_name', 'trange') 5936
('proc_name', 'file_name') 5202
('trange', 'file_name', 'proc_name') 5202
('trange', 'proc_name', 'file_name') 5202
('file_name', 'trange', 'proc_name') 5202
('file_name', 'proc_name', 'trange') 5202
('proc_name', 'trange', 'file_name') 5202
('proc_name', 'file_name', 'trange') 5202
time: 2.93 s (started: 2023-03-12 13:40:55 -07:00)


In [136]:
import dask.dataframe as dd
import pandas as pd
from dask import compute, delayed
from dask.distributed import get_client
from typing import Any, Dict

BOTTLENECK_ORDER = dict(
    file_name=('file_name', 'proc_name', 'trange'),
    proc_name=('proc_name', 'trange', 'file_name'),
    trange=('trange', 'proc_name', 'file_name'),
)
BOTTLENECK_TYPE = dict(
    file_name='file_name',
    proc_name='proc_name',
    trange='trange'
)

def _calculate_llc(level_row: pd.Series):
    llc = dict(level_row)
    for view_type in VIEW_TYPES:
        if view_type in llc:
            llc.pop(view_type)
        bottleneck_type = BOTTLENECK_TYPE[view_type]
        if bottleneck_type in llc:
            llc.pop(bottleneck_type)
    return llc


@delayed
def _process_bottleneck_view(
    view_key: tuple,
    threshold: float,
    low_level_view: pd.DataFrame,
    mid_level_view: pd.DataFrame,
    high_level_view: pd.DataFrame,
):
    # Get view type
    view_type = view_key[-1]
    # Get ordered bottleneck columns
    _, ml_col, ll_col = BOTTLENECK_ORDER[view_type]
    # Init bottlenecks
    bottlenecks = {}
    # Loop through index tuples
    ids_tuple = low_level_view.index
    for hl_id, ml_id, ll_id in ids_tuple:
        hl_row = high_level_view.loc[hl_id]
        ml_row = mid_level_view.loc[(hl_id, ml_id)]
        ll_row = low_level_view.loc[(hl_id, ml_id, ll_id)]
        if hl_id not in bottlenecks:
            bottlenecks[hl_id] = {}
            bottlenecks[hl_id]['llc'] = _calculate_llc(hl_row)
            bottlenecks[hl_id][ml_col] = {}
        if ml_id not in bottlenecks[hl_id][ml_col]:
            bottlenecks[hl_id][ml_col][ml_id] = {}
            bottlenecks[hl_id][ml_col][ml_id]['llc'] = _calculate_llc(ml_row)
            bottlenecks[hl_id][ml_col][ml_id][ll_col] = {}
        if ll_id not in bottlenecks[hl_id][ml_col][ml_id][ll_col]:
            bottlenecks[hl_id][ml_col][ml_id][ll_col][ll_id] = {}
            bottlenecks[hl_id][ml_col][ml_id][ll_col][ll_id]['llc'] = _calculate_llc(ll_row)
    # Return view key & bottlenecks
    return view_key, threshold, bottlenecks

def detect_bottlenecks(views: Dict[tuple, dd.DataFrame], view_types: list) -> Dict[tuple, object]:
    # Keep bottleneck views
    bottleneck_views = {}
    # Run through views
    for view_key, view in views.items():
        # Generate bottleneck views
        bottleneck_views[view_key] = _generate_bottlenecks_views(
            view_key=view_key,
            view=view,
            view_types=view_types,
            max_io_time=max_io_time,
            metric=metric,
        )
    # Generate bottlenecks
    bottlenecks = _process_bottleneck_views(
        bottleneck_views=bottleneck_views,
        metric=metric,
    )
    # Return bottleneck views
    return bottlenecks, bottleneck_views

def _process_bottleneck_views(bottleneck_views: Dict[tuple, dd.DataFrame], metric: str):
    # Init bottlenecks
    bottlenecks = {}
    bottlenecks_delayed = []
    # Run through bottleneck views
    for view_key, view_dict in bottleneck_views.items():
        # For given thresholds
        for th in DELTA_BINS[3:-1]:  # [0.25, 0.5, 0.75]

            threshold_col = f"{metric}_th"

            low_level_view = view_dict['low_level_view']
            mid_level_view = view_dict['mid_level_view']
            high_level_view = view_dict['high_level_view']

            bottlenecks_delayed.append(_process_bottleneck_view(
                view_key=view_key,
                threshold=th,
                low_level_view=low_level_view.query(f"{threshold_col} >= @th", local_dict={'th': th}),
                mid_level_view=mid_level_view.query(f"{threshold_col} >= @th", local_dict={'th': th}),
                high_level_view=high_level_view.query(f"{threshold_col} >= @th", local_dict={'th': th})
            ))

    # Compute all bottlenecks
    results = compute(*bottlenecks_delayed)
    for view_key, th, result in results:
        bottlenecks[view_key] = bottlenecks[view_key] if view_key in bottlenecks else {}
        bottlenecks[view_key][f"{th:.2f}"] = result
    # Return all bottlenecks
    return bottlenecks

def _generate_bottlenecks_views(
    view_key: tuple,
    view: dd.DataFrame,
    view_types: list,
    max_io_time: dd.core.Scalar,
    metric: str,
):
    # Get view type
    view_type = view_key[-1]
    bottleneck_type = BOTTLENECK_TYPE[view_type]

    # Create lower level view
    low_level_view = view \
        .groupby(list(BOTTLENECK_ORDER[view_type])) \
        .first() \
        .drop(columns=['acc_pat', 'io_cat', 'file_id', 'proc_id'], errors='ignore')

    # Non-proc agg columns
    non_proc_agg_dict = _get_agg_dict(view_types=view_types, view_columns=low_level_view.columns, is_proc=False)
    proc_agg_dict = _get_agg_dict(view_types=view_types, view_columns=low_level_view.columns, is_proc=True)

    # Create mid and high level views
    if bottleneck_type is not 'proc_name':
        mid_level_view = low_level_view \
            .reset_index() \
            .groupby([bottleneck_type, 'proc_name']) \
            .agg(non_proc_agg_dict)

        high_level_view = mid_level_view \
            .reset_index() \
            .groupby([bottleneck_type]) \
            .agg(proc_agg_dict)
    else:
        mid_level_view = low_level_view \
            .reset_index() \
            .groupby([bottleneck_type, 'trange']) \
            .agg(non_proc_agg_dict)

        high_level_view = mid_level_view \
            .reset_index() \
            .groupby([bottleneck_type]) \
            .agg(non_proc_agg_dict)

    low_level_view = low_level_view \
        .map_partitions(set_metric_percentages, metric=metric, max_io_time=max_io_time) \
        .map_partitions(set_metric_scores, metric=metric, col=f"{metric}_pero")

    mid_level_view = mid_level_view \
        .map_partitions(set_metric_percentages, metric=metric, max_io_time=max_io_time) \
        .map_partitions(set_metric_scores, metric=metric, col=f"{metric}_pero")

    high_level_view = high_level_view \
        .map_partitions(set_metric_percentages, metric=metric, max_io_time=max_io_time) \
        .map_partitions(set_metric_scores, metric=metric, col=f"{metric}_pero")

    return dict(
        low_level_view=low_level_view,
        mid_level_view=mid_level_view,
        high_level_view=high_level_view
    )

def _get_agg_dict(view_types: list, view_columns: list, is_proc=False):
    if is_proc:
        agg_dict = {col: max if any(x in col for x in 'duration time'.split()) else sum for col in view_columns}
    else:
        agg_dict = {col: sum for col in view_columns}
    # agg_dict['func_id'] = unique_flatten()
    agg_dict['size_min'] = min
    agg_dict['size_max'] = max
    for view_type in view_types:
        if view_type in agg_dict:
            agg_dict.pop(view_type)
        bottleneck_type = BOTTLENECK_TYPE[view_type]
        if bottleneck_type in agg_dict:
            agg_dict.pop(bottleneck_type)
    return agg_dict


time: 9.23 ms (started: 2023-03-12 13:40:59 -07:00)


In [137]:
bottlenecks, bottleneck_views = detect_bottlenecks(
    views=views,
    view_types=VIEW_TYPES,
)

len(bottlenecks)

15

time: 7.87 s (started: 2023-03-12 13:41:00 -07:00)


In [138]:
for view_key, bottleneck in bottlenecks.items():
    for th in bottleneck.keys():
        print(view_key, th, len(bottleneck[th]))

('trange',) 0.10 3
('trange',) 0.25 1
('trange',) 0.50 0
('trange',) 0.75 0
('file_name',) 0.10 10
('file_name',) 0.25 4
('file_name',) 0.50 0
('file_name',) 0.75 0
('proc_name',) 0.10 6
('proc_name',) 0.25 3
('proc_name',) 0.50 0
('proc_name',) 0.75 0
('trange', 'file_name') 0.10 10
('trange', 'file_name') 0.25 4
('trange', 'file_name') 0.50 0
('trange', 'file_name') 0.75 0
('trange', 'proc_name') 0.10 6
('trange', 'proc_name') 0.25 3
('trange', 'proc_name') 0.50 0
('trange', 'proc_name') 0.75 0
('file_name', 'trange') 0.10 3
('file_name', 'trange') 0.25 1
('file_name', 'trange') 0.50 0
('file_name', 'trange') 0.75 0
('file_name', 'proc_name') 0.10 6
('file_name', 'proc_name') 0.25 3
('file_name', 'proc_name') 0.50 0
('file_name', 'proc_name') 0.75 0
('proc_name', 'trange') 0.10 3
('proc_name', 'trange') 0.25 1
('proc_name', 'trange') 0.50 0
('proc_name', 'trange') 0.75 0
('proc_name', 'file_name') 0.10 10
('proc_name', 'file_name') 0.25 4
('proc_name', 'file_name') 0.50 0
('proc_name

In [139]:
bottleneck_views[('trange',)]['high_level_view'].compute()

,duration_sum,index_count,size_min,size_max,size_sum,read_count,write_count,metadata_count,read_size,write_size,...,data_size,data_time,open_time,close_time,seek_time,stat_time,duration_pero,duration_perr,duration_score,duration_th
trange,,,,,,,,,,,,,,,,,,,,,
1,6.923414,2919907,-2,178112,24761748419,2073527,802752,43628,9373804736,15386933803,...,24760738539,6.806267,0.181286,0.185384,0.013867,0.002602,0.816925,0.419713,very high,0.750
2,3.509167,1177132,-2,172352,13419947399,946066,225503,5563,5889112047,7530802998,...,13419915045,3.502166,0.050234,0.012108,0.000742,0.000412,0.414062,0.212734,medium,0.250
3,1.362043,283352,-2,172352,2662042086,175957,93913,13482,2279429306,382614336,...,2662043642,1.362043,0.012447,0.001577,0.000232,0.000357,0.160714,0.082570,low,0.100
4,2.218779,504757,-2,172352,2886798369,377903,124392,2462,2359928760,526870045,...,2886798805,2.203067,0.040549,0.005539,0.000187,0.000289,0.261804,0.134508,medium,0.250
5,0.319489,386712,-2,172352,2560132999,301038,83880,1794,2318578604,241554755,...,2560133359,0.319489,0.002262,0.001273,0.000204,0.000314,0.037698,0.019368,very low,0.010
6,0.301292,275635,-2,172352,2172873288,222893,52107,635,2022805300,150068160,...,2172873460,0.301292,0.002191,0.000392,0.000040,0.000010,0.035551,0.018265,very low,0.010
7,0.111530,166155,-2,172352,1623478810,97338,67786,1031,1428255426,195223680,...,1623479106,0.111530,0.009779,0.000605,0.000041,0.000010,0.013160,0.006761,very low,0.010
8,0.116256,120464,-2,172352,1321807824,34549,84666,1249,1077970096,243838080,...,1321808176,0.116256,0.002312,0.000600,0.000040,0.000010,0.013718,0.007048,very low,0.010
9,0.114024,115014,-2,172352,1218611873,20850,92874,1290,951135113,267477120,...,1218612233,0.114024,0.002397,0.028914,0.000042,0.000010,0.013454,0.006912,very low,0.010


time: 367 ms (started: 2023-03-12 13:41:18 -07:00)


In [140]:
bottlenecks[('trange',)]['0.50']

{}

time: 6.81 ms (started: 2023-03-12 13:41:33 -07:00)
